## 爬蟲完整版

### 引用package

In [4]:
import requests
from bs4 import BeautifulSoup
import math
import time

### 設定查詢的條件(看適用Post或者是Get)

In [46]:
#若為Post則需放入查詢的資料
payload = {
'method':'search',
'searchMethod':'true',
'tenderUpdate':'',
'searchTarget':'',
'orgName':'',
'orgId':'',
'hid_1':'1',
'tenderName':'',
'tenderId':'',
'tenderType':'tenderDeclaration',
'tenderWay':'1,2,3,4,5,6,7,10,12',
'tenderDateRadio':'on',
'tenderStartDateStr':'105/04/28',
'tenderEndDateStr':'105/05/04',
'tenderStartDate':'105/05/04',
'tenderEndDate':'105/05/04',
'isSpdt':'N',
'proctrgCate':'3',
'radProctrgCate':'3',
'btnQuery':'查詢',
'hadUpdated':''
}
rs = requests.session() #延續之前瀏覽過的sessionID
res = rs.post('http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic', data = payload)
#使用post方法將需要查詢的資料放入
soup = BeautifulSoup(res.text) #萃取查詢到的資料，並且用text文字格式呈現
#print soup
#print soup.select('.T11b')[0].text

### 抓取頁數

In [47]:
recnumber = int(soup.select('.T11b')[0].text) #將抓取到的文字內容，轉成為數值型態
pagenumber = int(math.ceil(float(recnumber) / 100))
#將recnumber變數的數值型態轉乘浮點數，並且除上100，使用ceil方法，無條件進入，並轉成數值型態。

## 標案連結存入檔案

In [48]:
urlformat = 'http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic&method=search&isSpdt=&pageIndex={}'
with open('C:\\Users\\BIG DATA\\Desktop\\bid_list.txt','w') as f:
    for i in range(1, pagenumber + 1):
        url = urlformat.format(i)
        res2 = rs.get(url)
        soup2 = BeautifulSoup(res2.text)
        print_area = soup2.select('#print_area')[0]
        for tr in print_area.select('tr')[1:-1]:
            f.write('http://web.pcc.gov.tw/tps' + tr.select('a')[0]['href'][2:] + '\n')

## 計算連結數量

In [49]:
cnt = 0
for line in open('C:\\Users\\BIG DATA\\Desktop\\bid_list.txt','r'):
    #print line 讀取每一行的連結資料
    cnt += 1
print cnt

#print len([line for line in open('C:\\Users\\BIG DATA\\Desktop\\bid_list.txt', 'r')])
#另外一種寫法

348


## 設定檔名

In [50]:
a = 'http://web.pcc.gov.tw/tps/tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&scope=F&primaryKey=51869557'
rearstring = a.split('&primaryKey=') #將切割後的資料，丟入rearstring，型態為List
print rearstring
primaryKey = rearstring[1]
print primaryKey

['http://web.pcc.gov.tw/tps/tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&scope=F', '51869557']
51869557


In [51]:
dic = {} #宣告一個字典
for bid_url in open('C:\\Users\\BIG DATA\\Desktop\\bid_list.txt','r'):
    #開啟檔案，並且宣告bid_url變數
    rearstring = bid_url.split('&primaryKey=') #將切割後的資料，丟入rearstring，型態為List
    primaryKey = rearstring[1]
    if primaryKey not in dic: #檢查若primarykey不在字典裡，則將value丟入相對應的key值
        dic[primaryKey] = bid_url
    else: #若有重複，則印出，value與key值
        print bid_url, dic[primaryKey]

In [53]:
for bid_url in open('C:\\Users\\BIG DATA\\Desktop\\bid_list.txt','r'):
    bid_detail = requests.get(bid_url.strip())
    soup = BeautifulSoup(bid_detail.text)
    printarea = soup.select('#print_area')
    rearstring = bid_url.strip().split('&primaryKey=')
    primaryKey = rearstring[1]
    with open('C:\\Users\\BIG DATA\\Desktop\\Data\\{}.txt'.format(primaryKey),'w') as f:
        #prettify是轉換bs4套件物件為字串
        f.write(printarea[0].prettify('utf-8'))
    time.sleep(5)

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

## 使用File IO讀取資料

In [4]:
with open('C:\\Users\\BIG DATA\\Desktop\\Data\\51868739.txt', 'r') as f:
    print f.read()
    #讀取資料

<div id="print_area">
 <input id="fkPmsAwardWay" type="hidden" value="1"/>
 <input id="isLowestLaw" type="hidden" value="N"/>
 <table border="1" cellpadding="0" cellspacing="0" class="table_block tender_table" summary="金門縣採購招標所之招標公告，公告日為105/05/04" width="772">
  <caption style="display:none">
   <center>
    <h1>
     公開取得報價單或企劃書公告
    </h1>
    <h3>
     公告日：105/05/04
    </h3>
   </center>
  </caption>
  <tr>
   <th align="center" colspan="3" style="border:0px solid #FFFFFF;border:1px solid #FFFFFF;border-bottom:1px solid #89c0c7;">
    <center>
     <h1>
      公開取得報價單或企劃書公告
     </h1>
     <h3>
      公告日：105/05/04
     </h3>
    </center>
   </th>
  </tr>
  <tr class="tender_table_tr_1">
   <td bgcolor="#89c0c7" class="T11b" rowspan="8" style="text-align:center" valign="middle" width="5%">
    機
    <br/>
    關
    <br/>
    資
    <br/>
    料
   </td>
   <th bgcolor="#DAEBED" class="T11b" width="25%">
    機關代碼
   </th>
   <td bgcolor="#EFF1F1" class="newstop" width="75%">
    3.71.2

## 使用bs4萃取資料

In [18]:
from bs4 import BeautifulSoup
dic = {
    u'機關代碼':'',
    u'機關名稱':'',
    u'單位名稱':'',
    u'機關地址':'',
    u'聯絡人':'',
    u'聯絡電話':'',
    u'傳真號碼':'',
    u'電子郵件信箱':'',
    u'標案案號':'',
    u'案名稱':'',
    u'標的分類':'',
    u'財物採購性質':'',
    u'採購金額級距':'',
    u'辦理方式':'',
    u'依據法條':'',
    u'預算金額':'',
    u'招標方式':'',
    u'決標方式':''
}
with open('C:\\Users\\Albert\\Desktop\\51868739.txt', 'r') as f: #匯入檔案
    soup = BeautifulSoup(f.read())
    tableStr = "tr.tender_table_tr_" #因為tr.tender_table_tr_x有規律性，而x為數字，故用字串型式
    for index in range(1,4): #承上關係，故使用for迴圈，自動代入
        selectNum = tableStr + str(index) #將int資料型態轉為字串，並且組合
        for element in soup.select(selectNum):
                print element.select('th.T11b')[0].text.strip(), element.select('td.newstop')[0].text.strip()

機關代碼 3.71.2.63
機關名稱 金門縣採購招標所
單位名稱 金門縣採購招標所
機關地址 893金門縣金城鎮民權路72號
聯絡人 莊子箳
聯絡電話 (082)373257分機312
傳真號碼 (082)325935
電子郵件信箱 tzeping1@mail.kinmen.gov.tw
標案案號 105210230159
標案名稱 金門縣衛生局105年度市售農藥殘留衛生安全抽驗計畫（單價決標）
標的分類 勞務類
    
    94
				-
				污水及垃圾處理、公共衛生及其他環保服務
財物採購性質 非屬財物之工程或勞務
採購金額級距 未達公告金額
辦理方式 代辦
				
					
					，洽辦機關：
					3.71.1.37
					
					金門縣衛生局
依據法條 採購法第49條
預算金額 252,000元
預算金額是否公開 是
後續擴充 是
    

     依政府採購法第22條第1項第7款，須敘明後續擴充之期間、金額或數量：
     
     本採購預計委託辦理期限至105年10月31日止，機關並保留依原決標契約價金及條件擴充採購6個月，以換文（約）方式辦理，免召開議價會議及程序，擴充之金額約新台幣54,000元整。
是否受機關補助 否
是否含特別預算 否
招標方式 公開取得報價單或企劃書
決標方式 最低標
是否依政府採購法施行細則第64條之2辦理 否
是否電子報價 否
新增公告傳輸次數 01
招標狀態 第一次公開取得
公告日 105/05/04
是否複數決標 否
是否訂有底價 是
是否屬特殊採購 否
是否已辦理公開閱覽 否
是否屬統包 否
是否屬共同供應契約採購 否
是否屬二以上機關之聯合採購(不適用共同供應契約規定) 否
是否應依公共工程專業技師簽證規則實施技師簽證 否
是否採行協商措施 否
是否適用採購法第104條或105條或招標期限標準第10條或第4條之1 否
是否依據採購法第106條第1項第1款辦理 否


## 字體格式上處理

In [36]:
from bs4 import BeautifulSoup
dic = {
    u'機關代碼':'',
    u'機關名稱':'',
    u'單位名稱':'',
    u'機關地址':'',
    u'聯絡人':'',
    u'聯絡電話':'',
    u'傳真號碼':'',
    u'電子郵件信箱':'',
    u'標案案號':'',
    u'標案名稱':'',
    u'標的分類':'',
    u'財物採購性質':'',
    u'採購金額級距':'',
    u'辦理方式':'',
    u'依據法條':'',
    u'預算金額':'',
    u'招標方式':'',
    u'決標方式':''
}
with open('C:\\Users\\Albert\\Desktop\\51868739.txt', 'r') as f: #匯入檔案
    soup = BeautifulSoup(f.read())
    tableStr = "tr.tender_table_tr_" #因為tr.tender_table_tr_x有規律性，而x為數字，故用字串型式
    for index in range(1,4): #承上關係，故使用for迴圈，自動代入
        selectNum = tableStr + str(index) #將int資料型態轉為字串，並且組合
        for element in soup.select(selectNum):
                if element.select('th.T11b')[0].text.strip() in dic:
                #若key在dic內的話
                    dic[element.select('th.T11b')[0].text.strip()] = element.select('td.newstop')[0].text.strip()
                    #將value丟入dic[key]內

## Print出看dic內的內容

In [37]:
for ele in dic:
    print ele, dic[ele]

電子郵件信箱 tzeping1@mail.kinmen.gov.tw
財物採購性質 非屬財物之工程或勞務
標的分類 勞務類
    
    94
				-
				污水及垃圾處理、公共衛生及其他環保服務
辦理方式 代辦
				
					
					，洽辦機關：
					3.71.1.37
					
					金門縣衛生局
預算金額 252,000元
聯絡電話 (082)373257分機312
招標方式 公開取得報價單或企劃書
採購金額級距 未達公告金額
機關地址 893金門縣金城鎮民權路72號
標案案號 105210230159
決標方式 最低標
機關代碼 3.71.2.63
標案名稱 金門縣衛生局105年度市售農藥殘留衛生安全抽驗計畫（單價決標）
傳真號碼 (082)325935
聯絡人 莊子箳
單位名稱 金門縣採購招標所
依據法條 採購法第49條
機關名稱 金門縣採購招標所


## 修正字串的格式

In [38]:
for ele in dic:
    print ele, repr(dic[ele]) #現出原樣

電子郵件信箱 u'tzeping1@mail.kinmen.gov.tw'
財物採購性質 u'\u975e\u5c6c\u8ca1\u7269\u4e4b\u5de5\u7a0b\u6216\u52de\u52d9'
標的分類 u'\u52de\u52d9\u985e\n    \n    94\r\n\t\t\t\t-\r\n\t\t\t\t\u6c61\u6c34\u53ca\u5783\u573e\u8655\u7406\u3001\u516c\u5171\u885b\u751f\u53ca\u5176\u4ed6\u74b0\u4fdd\u670d\u52d9'
辦理方式 u'\u4ee3\u8fa6\r\n\t\t\t\t\r\n\t\t\t\t\t\r\n\t\t\t\t\t\uff0c\u6d3d\u8fa6\u6a5f\u95dc\uff1a\r\n\t\t\t\t\t3.71.1.37\r\n\t\t\t\t\t\r\n\t\t\t\t\t\u91d1\u9580\u7e23\u885b\u751f\u5c40'
預算金額 u'252,000\u5143'
聯絡電話 u'(082)373257\u5206\u6a5f312'
招標方式 u'\u516c\u958b\u53d6\u5f97\u5831\u50f9\u55ae\u6216\u4f01\u5283\u66f8'
採購金額級距 u'\u672a\u9054\u516c\u544a\u91d1\u984d'
機關地址 u'893\u91d1\u9580\u7e23\u91d1\u57ce\u93ae\u6c11\u6b0a\u8def72\u865f'
標案案號 u'105210230159'
決標方式 u'\u6700\u4f4e\u6a19'
機關代碼 u'3.71.2.63'
標案名稱 u'\u91d1\u9580\u7e23\u885b\u751f\u5c40105\u5e74\u5ea6\u5e02\u552e\u8fb2\u85e5\u6b98\u7559\u885b\u751f\u5b89\u5168\u62bd\u9a57\u8a08\u756b\uff08\u55ae\u50f9\u6c7a\u6a19\uff09'
傳真號碼 u'(082)325935'
聯絡人 u'\u

In [39]:
for ele in dic:
    print ele, ' '.join(dic[ele].split()) #修正後的模樣

電子郵件信箱 tzeping1@mail.kinmen.gov.tw
財物採購性質 非屬財物之工程或勞務
標的分類 勞務類 94 - 污水及垃圾處理、公共衛生及其他環保服務
辦理方式 代辦 ，洽辦機關： 3.71.1.37 金門縣衛生局
預算金額 252,000元
聯絡電話 (082)373257分機312
招標方式 公開取得報價單或企劃書
採購金額級距 未達公告金額
機關地址 893金門縣金城鎮民權路72號
標案案號 105210230159
決標方式 最低標
機關代碼 3.71.2.63
標案名稱 金門縣衛生局105年度市售農藥殘留衛生安全抽驗計畫（單價決標）
傳真號碼 (082)325935
聯絡人 莊子箳
單位名稱 金門縣採購招標所
依據法條 採購法第49條
機關名稱 金門縣採購招標所
